In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import f1_score, mean_squared_error as mse
from sklearn import svm
from sklearn.linear_model import LogisticRegression

def ignore_warnings():
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=Warning)

ignore_warnings()

In [2]:
data = pd.read_csv('clean_data/players_17_clean_train.csv')
X1_train= data.iloc[:, :-1].values
y1_train = data.iloc[:, -1].values
s1 = data.iloc[:, -1].values
c1 = Counter(s1)[1]/Counter(y1_train)[1]

data = pd.read_csv('clean_data/players_17_clean_test.csv')
X1_test= data.iloc[:, :-1].values
y1_test = data.iloc[:, -1].values

In [34]:
def frequency(i, j, collection):
    count = 0
    for elem in collection:
        if elem[i] == j:
            count += 1
    return count

def dnf(train_x, train_y):
    num_rows, num_cols = train_x.shape
    PF = np.zeros(shape=(0, 2))
    
    RN = np.zeros(shape=(0, num_cols))
    RN_y = np.array([])
    U = np.zeros(shape=(0, num_cols))
    U_y = np.array([])
    P = np.zeros(shape=(0, num_cols))
    P_y = np.array([])
    
    # RN = U (RN: reliable negative documents)
    for i in range(0, len(train_y)-1):
        if (train_y[i] == 0):
            RN = np.vstack([RN, train_x[i]])
            RN_y = np.append(RN_y, train_y[i])
            U = np.vstack([U, train_x[i]])
            U_y = np.append(U_y, train_y[i])
        else:
            P = np.vstack([P, train_x[i]])
            P_y = np.append(P_y, train_y[i])
            
    for i in range(0, num_cols-1):
        for j in range(0, num_rows-1):
            symbol = train_x[j][i]
            constraint_1 = frequency(i, symbol, P) / len(P)
            constraint_2 = frequency(i, symbol, U) / len(U)
            if (constraint_1 > constraint_2) and (constraint_1 > 0.5):
                new = [i, symbol]
                PF = np.vstack([PF, new])
    index = []
                
    for elem in range(0, len(train_x)-1):
        for constraint in PF:
            element = train_x[elem]
            if element[int(constraint[0])] == constraint[1]:
                index.append(elem)
                
    RN = np.delete(RN, index)
    
    #remove duplicates
    PF = np.unique(PF)
    
    return P, P_y, U_y, RN, RN_y, PF

In [35]:
P, P_y, U_y, RN, RN_y, PF = dnf(X1_train, y1_train)
print(PF)

[0. 2.]


In [33]:
from sklearn.svm import SVC

training_x = np.vstack([P, RN])
training_y = np.vstack([P_y, RN_y])

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(training_x, training_y)

y_pred = clf.predict(np.copy(X1_test))
            
print(name,"F1 score:", f1_score(y1_test, y_pred))
#print(name,"MSE score:", mse(y_best_prob_1, y_pred_prob))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 22 and the array at index 1 has size 233793

In [36]:
print(RN)

[0.         0.5        0.69230769 ... 0.48780488 0.45744681 0.46666667]
